In [16]:
import pandas as pd
#import tensorflow as tf

In [17]:
dataset = pd.read_csv('netflix_titles.csv')

In [18]:
dataset.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."


In [4]:
dataset.shape

(6234, 12)

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6234 entries, 0 to 6233
Data columns (total 12 columns):
show_id         6234 non-null int64
type            6234 non-null object
title           6234 non-null object
director        4265 non-null object
cast            5664 non-null object
country         5758 non-null object
date_added      6223 non-null object
release_year    6234 non-null int64
rating          6224 non-null object
duration        6234 non-null object
listed_in       6234 non-null object
description     6234 non-null object
dtypes: int64(2), object(10)
memory usage: 584.6+ KB


# Cleaning Data

In [19]:
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import io
import numpy as np

In [20]:
original_len = len(dataset)
dropped_df = dataset.dropna()
dropped_len = len(dropped_df)
print("% of rows with missing values: " + str((original_len - dropped_len) / dropped_len * 100) + '%')
print()
print("Number of null values in each column: ")
print(dataset.isnull().sum())
dataset_clean = dropped_df

% of rows with missing values: 65.18282988871225%

Number of null values in each column: 
show_id            0
type               0
title              0
director        1969
cast             570
country          476
date_added        11
release_year       0
rating            10
duration           0
listed_in          0
description        0
dtype: int64


In [21]:
df = dataset_clean.drop(columns=['show_id', 'date_added', 'duration', 'description'])

In [23]:
df

,type,title,director,cast,country,release_year,rating,listed_in
0,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",2019,TV-PG,"Children & Family Movies, Comedies"
4,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,2017,TV-14,Comedies
6,Movie,Automata,Gabe Ibáñez,"Antonio Banderas, Dylan McDermott, Melanie Gri...","Bulgaria, United States, Spain, Canada",2014,R,"International Movies, Sci-Fi & Fantasy, Thrillers"
7,Movie,Fabrizio Copano: Solo pienso en mi,"Rodrigo Toro, Francisco Schultz",Fabrizio Copano,Chile,2017,TV-MA,Stand-Up Comedy
9,Movie,Good People,Henrik Ruben Genz,"James Franco, Kate Hudson, Tom Wilkinson, Omar...","United States, United Kingdom, Denmark, Sweden",2014,R,"Action & Adventure, Thrillers"
...,...,...,...,...,...,...,...,...
6142,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,2019,TV-PG,"British TV Shows, Reality TV"
6158,TV Show,Miraculous: Tales of Ladybug & Cat Noir,Thomas Astruc,"Cristina Vee, Bryce Papenbrook, Keith Silverst...","France, South Korea, Japan",2018,TV-Y7,"Kids' TV, TV Action & Adventure"
6167,TV Show,Sacred Games,"Vikramaditya Motwane, Anurag Kashyap","Saif Ali Khan, Nawazuddin Siddiqui, Radhika Ap...","India, United States",2019,TV-MA,"Crime TV Shows, International TV Shows, TV Dramas"
6182,TV Show,Men on a Mission,Jung-ah Im,"Ho-dong Kang, Soo-geun Lee, Sang-min Lee, Youn...",South Korea,2019,TV-14,"International TV Shows, Korean TV Shows, Stand..."


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3774 entries, 0 to 6213
Data columns (total 8 columns):
type            3774 non-null object
title           3774 non-null object
director        3774 non-null object
cast            3774 non-null object
country         3774 non-null object
release_year    3774 non-null int64
rating          3774 non-null object
listed_in       3774 non-null object
dtypes: int64(1), object(7)
memory usage: 265.4+ KB


In [25]:
df['country'] = df['country'].map(lambda x: x.split(',')[0])

bag_of_words_data = ['director', 'cast', 'listed_in']

for col in bag_of_words_data:
    df[col] = df[col].map(lambda x : x.lower().replace(' ', '').split(',')[:3])

df['bag_of_words'] = ''
for i, row in df.iterrows():
    words = [' '.join(row[col]) for col in bag_of_words_data]
    df.loc[i, 'bag_of_words'] = ' '.join(words)
    
df.drop(columns=bag_of_words_data, inplace=True)
df.head()

,type,title,country,release_year,rating,bag_of_words
0,Movie,Norm of the North: King Sized Adventure,United States,2019,TV-PG,richardfinn timmaltby alanmarriott andrewtoth ...
4,Movie,#realityhigh,United States,2017,TV-14,fernandolebrija nestacooper katewalsh johnmich...
6,Movie,Automata,Bulgaria,2014,R,gabeibáñez antoniobanderas dylanmcdermott mela...
7,Movie,Fabrizio Copano: Solo pienso en mi,Chile,2017,TV-MA,rodrigotoro franciscoschultz fabriziocopano st...
9,Movie,Good People,United States,2014,R,henrikrubengenz jamesfranco katehudson tomwilk...


# One hot-encode

In [12]:
df= pd.get_dummies(dataset, columns=['type', 'rating', 'country'])
df.reset_index(drop=True, inplace=True)

In [15]:
df.head()

,show_id,title,director,cast,date_added,release_year,duration,listed_in,description,type_Movie,...,"country_United States, United Kingdom, Spain, South Korea","country_United States, Uruguay","country_United States, Venezuela",country_Uruguay,"country_Uruguay, Argentina, Spain","country_Uruguay, Spain, Mexico",country_Venezuela,"country_Venezuela, Colombia",country_Vietnam,country_West Germany
0,81145628,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","September 9, 2019",2019,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,1,...,0,0,0,0,0,0,0,0,0,0
1,80117401,Jandino: Whatever it Takes,NaN,Jandino Asporaat,"September 9, 2016",2016,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...,1,...,0,0,0,0,0,0,0,0,0,0
2,70234439,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...","September 8, 2018",2013,1 Season,Kids' TV,"With the help of three human allies, the Autob...",0,...,0,0,0,0,0,0,0,0,0,0
3,80058654,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...","September 8, 2018",2016,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...,0,...,0,0,0,0,0,0,0,0,0,0
4,80125979,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...","September 8, 2017",2017,99 min,Comedies,When nerdy high schooler Dani finally attracts...,1,...,0,0,0,0,0,0,0,0,0,0


In [14]:
vectorizer = CountVectorizer()
count_matrix = vectorizer.fit_transform(df['bag_of_words'])

similarities = cosine_similarity(count_matrix, count_matrix)
dissimilarities = 1 - similarities

KeyError: 'bag_of_words'

# Create a Model

In [ ]:
amountOfUsedUsers = 1000

trX = []

for user, curUser in 

In [ ]:
dataset.shape

# Data Information

In [ ]:
import matplotlib.pyplot as plt

### Based on type

In [ ]:
based_on_type=dataset.type.value_counts()
based_on_type.plot.pie(autopct='%1.1f%%')
plt.show()

I assume we cannot clean the data based on null value, because it will be bad dataset. Do you have any idea?

### Based on Country

In [ ]:
country_ = []

for initial in dataset.country:
    initial = str(initial)
    country = initial.strip('').split(', ')
    for k in country:
        country_.append(k)

country_series=pd.Series(country_)
based_on_country=country_series.value_counts().head(15)
based_on_country.plot.barh()
plt.title('Country')
plt.show()

print(based_on_country)

### Based on date

In [ ]:
dataset["date_added"] = pd.to_datetime(dataset['date_added'])
dataset['year_added'] = dataset['date_added'].dt.year

movie_per_year=[]
tv_shows_per_year=[]

for i in range(2008,2020):
    h=dataset.loc[(dataset['type']=='Movie') & (dataset.year_added==i)] 
    g=dataset.loc[(dataset['type']=='TV Show') & (dataset.year_added==i)] 
    movie_per_year.append(len(h))
    tv_shows_per_year.append(len(g))

mylist= list(range(2008,2020))

opacity = 0.4
bar_width = 0.35
barlist=[0.35]*12
mynewlist=[x + y for x, y in zip(mylist, barlist)]

plt.xlabel('Year')
plt.ylabel('Total')

plt.xticks(mylist, rotation=30)
bar1 = plt.bar(mylist, movie_per_year, bar_width, align='center', alpha=opacity, color='b', label='movie_per_year')
bar2 = plt.bar(mynewlist, tv_shows_per_year, bar_width, align='center', alpha=opacity, color='r',label='tv_shows_per_year')

# Add counts above the two bar graphs
for rect in bar1 + bar2:
    height = rect.get_height()
    plt.text(rect.get_x() + rect.get_width()/2.0, height, '%d' % int(height), ha='center', va='bottom')

plt.legend()
plt.tight_layout()
plt.plot(mylist, movie_per_year, label='Movie',color='b')
plt.plot(mynewlist, tv_shows_per_year, label='TV Shows',color='r')
plt.scatter(mylist, movie_per_year, label='Movie',color='b')
plt.scatter(mynewlist, tv_shows_per_year, label='TV Shows',color='r')
plt.show()

### Based on release_year

In [ ]:
movie_release_year=[]
tv_shows_release_year=[]

for i in range(2005,2020):
    h=dataset.loc[(dataset['type']=='Movie') & (dataset.release_year==i)] 
    g=dataset.loc[(dataset['type']=='TV Show') & (dataset.release_year==i)] 
    movie_release_year.append(len(h))
    tv_shows_release_year.append(len(g))

mylist= list(range(2005,2020))

opacity = 0.4
bar_width = 0.3
barlist=[bar_width]*len(mylist)
mynewlist=[x + y for x, y in zip(mylist, barlist)]

plt.xlabel('Year')
plt.ylabel('Total')

plt.xticks(mylist, rotation=30)
bar1 = plt.bar(mylist, movie_release_year, bar_width, align='center', alpha=opacity, color='b', label='movie_release_year')
bar2 = plt.bar(mynewlist, tv_shows_release_year, bar_width, align='center', alpha=opacity, color='r',label='tv_shows_release_year')

# Add counts above the two bar graphs
for rect in bar1 + bar2:
    height = rect.get_height()
    plt.text(rect.get_x() + rect.get_width()/2.0, height, '%d' % int(height), ha='center', va='bottom')

plt.legend()
plt.tight_layout()
plt.plot(mylist, movie_release_year, label='Movie',color='b')
plt.plot(mynewlist, tv_shows_release_year, label='TV Shows',color='r')
plt.scatter(mylist, movie_release_year, label='Movie',color='b')
plt.scatter(mynewlist, tv_shows_release_year, label='TV Shows',color='r')
plt.show()

### Based on Rating

In [ ]:
based_on_type=dataset.rating.value_counts()
based_on_type.plot.bar()
plt.xticks(rotation=30)
plt.show()

### Based On Duration

source kaggle.com

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

countries_0_7=['United States','Australia','Turkey','Hong Kong','Thailand',"United Kingdom",'Taiwan']
countrie_7_14=['France','Spain'
          ,'Mexico','Japan','South Korea','India','Canada']
def country_mov_dur(country):
    netflix_country_dur=dataset.loc[(dataset.country==str(country))&(dataset.type=='Movie')].duration[:]
    # # ' minSeaso' will remove ' Season' and ' min'
    netflix_country_dur=netflix_country_dur.apply(lambda x : int(x.strip(' minSeaso')))
    
    return netflix_country_dur
f, axes = plt.subplots(7,1,figsize=(18,18))
for i in range(7):
    for j in range(1):
        country_mov_duration=country_mov_dur(str(countries_0_7[i]))
        
        sns.kdeplot(country_mov_duration,Label='Movie Duration'+' in '+str(countries_0_7[i]),color='g',ax=axes[i])
        
f, axes = plt.subplots(7,1,figsize=(18,18))
for i in range(7):
    for j in range(1):
        country_mov_duration=country_mov_dur(str(countrie_7_14[i]))
        
        sns.kdeplot(country_mov_duration,Label='Movie Duration'+' in '+str(countrie_7_14[i]),color='g',ax=axes[i])

### Listed_in

In [ ]:
listed_in_ = []

for initial in dataset.listed_in:
    initial = str(initial)
    listed_in = initial.strip('').split(', ')
    for k in listed_in:
        listed_in_.append(k)

listed_in_series=pd.Series(listed_in_)
based_on_listed_in=listed_in_series.value_counts().head(15)
based_on_listed_in.plot.barh()
plt.title('Listed_in')
plt.show()

print(based_on_listed_in)